# Unit 2: RAG, Vector Stores, and Indexing

In [1]:
!pip install -U langchain langchain-huggingface sentence-transformers langchain-community transformers accelerate python-dotenv

  Using cached transformers-5.2.0-py3-none-any.whl.metadata (32 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-5.1.0-py3-none-any.whl.metadata (31 kB)
  Using cached transformers-5.0.0-py3-none-any.whl.metadata (37 kB)


In [2]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables if you have a .env file
load_dotenv()

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'} # Change to 'cuda' if using a GPU runtime
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
vector = embeddings.embed_query("Apple")

print(f"Dimensionality: {len(vector)}")
print(f"First 5 numbers: {vector[:5]}")

Dimensionality: 384
First 5 numbers: [-0.006138464901596308, 0.031011823564767838, 0.06479359418153763, 0.010941491462290287, 0.005267174914479256]


## 3. The Math: Cosine Similarity

In [4]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

vec_cat = embeddings.embed_query("Cat")
vec_dog = embeddings.embed_query("Dog")
vec_car = embeddings.embed_query("Car")

print(f"Cat vs Dog: {cosine_similarity(vec_cat, vec_dog):.4f}")
print(f"Cat vs Car: {cosine_similarity(vec_cat, vec_car):.4f}")

Cat vs Dog: 0.6606
Cat vs Car: 0.4633


# Unit 2 - Part 4b: Naive RAG Pipeline

## 1. Introduction: The Open-Book Test

RAG (Retrieval-Augmented Generation) is just an Open-Book Test architecture.
1.  **Retrieval:** Find the right page in the textbook.
2.  **Generation:** Write the answer using that page.

### The Pipeline (Flowchart)
```mermaid
graph TD
    User[User Question] --> Retriever[Retriever System]
    Retriever -->|Search Database| Docs[Relevant Documents]
    Docs --> Combiner[Prompt Template]
    User --> Combiner
    Combiner -->|Full Prompt w/ Context| LLM[Gemini Model]
    LLM --> Answer[Final Answer]
```

In [7]:
# Setup
!pip install python-dotenv faiss-cpu langchain-huggingface sentence-transformers langchain-community langchain_google_genai
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Using the same free model as Part 4a
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.4 MB/s eta 0:00:00
Enter your Google API Key: ··········


## 2. The "Knowledge Base" (Grounding)

In [9]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Piyush's favorite food is Pizza with extra cheese."),
    Document(page_content="The secret password to the lab is 'Blueberry'."),
    Document(page_content="LangChain is a framework for developing applications powered by language models."),
]

## 3. Indexing ( Storing the knowledge)

In [10]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

## 4. The RAG Chain

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
Answer based ONLY on the context below:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("What is the secret password?")
print(result)

The secret password to the lab is 'Blueberry'.


## 5. Analysis

The retrieval step is opaque here. In the next notebook (**4c**), we will look *inside* the retriever to understand how FAISS actually finds that document among millions of others.

# Unit 2 - Part 4c: Deep Dive into Indexing Algorithms

## 1. Introduction: The Scale Problem

In [12]:
import faiss
import numpy as np

# Mock Data: 10,000 vectors of size 128
d = 128
nb = 10000
xb = np.random.random((nb, d)).astype('float32')

## 2. Flat Index (Brute Force)

**Concept:** Check every single item.

- **Algo:** `IndexFlatL2`
- **Pros:** 100% Accuracy (Gold Standard).
- **Cons:** Slow (O(N)). Unusable at 1M+ vectors.

In [13]:
index = faiss.IndexFlatL2(d)
index.add(xb)
print(f"Flat Index contains {index.ntotal} vectors")

Flat Index contains 10000 vectors


## 3. IVF (Inverted File Index)

**Concept:** Clustering / Partitioning.

Imagine looking for a book. Instead of checking every shelf, you go to the "Sci-Fi" section. Then you only search books *in that section*.


In [14]:
nlist = 100 # How many 'zip codes' (clusters) we want
quantizer = faiss.IndexFlatL2(d) # The calculator for distance
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)

# We MUST train it first so it learns where the clusters are
index_ivf.train(xb)
index_ivf.add(xb)


## 4. HNSW (Hierarchical Navigable Small World)

In [15]:
M = 16 # Number of connections per node (The 'Hub' factor)
index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.add(xb)

## 5. PQ (Product Quantization)

In [16]:
m = 8 # Split vector into 8 sub-vectors
index_pq = faiss.IndexPQ(d, m, 8)
index_pq.train(xb)
index_pq.add(xb)
print("PQ Compression complete. RAM usage minimized.")

PQ Compression complete. RAM usage minimized.
